# Lab Journal Project 1. Analysis of Ecoli antibiotic resistance

* [Data loading](#first-bullet)
* [Data analysis](#second-bullet)
* [Filtering reads](#third-bullet)
* [Aligment](#fourth-bullet)
* [Sort and index BAM file](#fifth-bullet)
* [Variant finding](#sixth-bullet)
* [Variants annotation](#seventh-bullet)

## Data loading <a class="anchor" id="first-bullet"></a>

In [1]:
!mkdir data
!ls

Downloading reference E.coli files to data path. More information about E.coli https://www.ncbi.nlm.nih.gov/genome/167
- *.fna - FASTA format of the nucleotide sequences
- *.gff - annotation of the genomic sequence in GFF3
- *.gbff - GenBank flat file format for the assembly

In [2]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz -O data/GCF_000005845.2_ASM584v2_genomic.fna.gz
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gff.gz -O data/GCF_000005845.2_ASM584v2_genomic.gff.gz
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz -O data/GCF_000005845.2_ASM584v2_genomic.gbff.gz
    

In [7]:
!gunzip data/GCF_000005845.2_ASM584v2_genomic.fna.gz
!gunzip data/GCF_000005845.2_ASM584v2_genomic.gff.gz
!gunzip data/GCF_000005845.2_ASM584v2_genomic.gbff.gz

In [8]:
!head -10 data/GCF_000005845.2_ASM584v2_genomic.fna

>NC_000913.3 Escherichia coli str. K-12 substr. MG1655, complete genome
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTG
GTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGAC
AGATAAAAATTACAGAGTACACAACATCCATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGT
AACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCCCGCACCTGACAGTGCGGGCTTTTTTTTTCGACCAAAGG
TAACGAGGTAACAACCATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCG
ATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTG
GCGATGATTGAAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTT
GACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAA
AACATGTCCTGCATGGCATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGCTGATTTGCCGTGGCGAGAAA


In [9]:
!head -12 data/GCF_000005845.2_ASM584v2_genomic.gff

##gff-version 3
#!gff-spec-version 1.21
#!processor NCBI annotwriter
#!genome-build ASM584v2
#!genome-build-accession NCBI_Assembly:GCF_000005845.2
##sequence-region NC_000913.3 1 4641652
##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=511145
NC_000913.3	RefSeq	region	1	4641652	.	+	.	ID=NC_000913.3:1..4641652;Dbxref=taxon:511145;Is_circular=true;Name=ANONYMOUS;gbkey=Src;genome=chromosome;mol_type=genomic DNA;strain=K-12;substrain=MG1655
NC_000913.3	RefSeq	gene	190	255	.	+	.	ID=gene-b0001;Dbxref=ASAP:ABE-0000006,ECOCYC:EG11277,GeneID:944742;Name=thrL;gbkey=Gene;gene=thrL;gene_biotype=protein_coding;gene_synonym=ECK0001;locus_tag=b0001
NC_000913.3	RefSeq	CDS	190	255	.	+	0	ID=cds-NP_414542.1;Parent=gene-b0001;Dbxref=UniProtKB/Swiss-Prot:P0AD86,Genbank:NP_414542.1,ASAP:ABE-0000006,ECOCYC:EG11277,GeneID:944742;Name=NP_414542.1;gbkey=CDS;gene=thrL;locus_tag=b0001;orig_transcript_id=gnl|b0001|mrna.NP_414542;product=thr operon leader peptide;protein_id=NP_414542.1;transl_

Downloading raw Illumina sequencing reads from shotgun sequencing of an E. coli strain that is resistant to the antibiotic _ampicillin_ 

In [10]:
!wget https://figshare.com/ndownloader/articles/10006541/versions/3 -O data/amp_res.zip

In [11]:
!unzip data/amp_res.zip -d data
!ls data

In [12]:
!gunzip data/amp_res_1.fastq.gz
!gunzip data/amp_res_2.fastq.gz
!ls data

In [13]:
!head -10 data/amp_res_1.fastq

@SRR1363257.37 GWZHISEQ01:153:C1W31ACXX:5:1101:14027:2198 length=101
GGTTGCAGATTCGCAGTGTCGCTGTTCCAGCGCATCACATCTTTGATGTTCACGCCGTGGCGTTTAGCAATGCTTGAAAGCGAATCGCCTTTGCCCACACG
+
@?:=:;DBFADH;CAECEE@@E:FFHGAE4?C?DE<BFGEC>?>FHE4BFFIIFHIBABEECA83;>>@>@CCCDC9@@CC08<@?@BB@9:CC#######
@SRR1363257.46 GWZHISEQ01:153:C1W31ACXX:5:1101:19721:2155 length=101
GTATGAGGTTTTGCTGCATTCTCTGNGCGAATATTAACTCCNTNNNNNTTATAGTTCAAAGCAAGTACCTGTCTCTTATACACATCTCCGAGCCCACGAGC
+
@@<?=D?D==?<AFGDF+AIHEACH#22<:?E8??:9??GG#0#####000;CF=C)4.==CA@@@)=7?C7?E37;3@>;;(.;>AB#############
@SRR1363257.77 GWZHISEQ01:153:C1W31ACXX:5:1101:5069:2307 length=101
GCTTCTCTTAACTGAGGTCACCATCATGCCGTTAAGTCCCTACCTCTCTTTTGCCGGTAACTGTTCCGCCGCGATTGCCTTTTATCTGTCTCTTATACACC


In [14]:
!head -10 data/amp_res_2.fastq

@SRR1363257.37 GWZHISEQ01:153:C1W31ACXX:5:1101:14027:2198 length=101
GATCTAAGCTGAAGCCAGGCCAAAGTTTGACGATTGGTGCAGGCAGTAGCGCACAGCGACTGGCAAACAACAGCGATAGCATTACGTATCGTGTGCGCAAA
+
???BDB:DFHBFD@9;;+A;AFGH;ABHFHHGE@9:B:??@D>@;F?D8<<F8AA9EHHD8'..;5?A?A992(',(59CC3@C>22::A238+2>B<>B<
@SRR1363257.46 GWZHISEQ01:153:C1W31ACXX:5:1101:19721:2155 length=101
GTACTTGCTTTGNACTATAATATGCACGGAGNTAATATTCGCTCAGAGAATGCAGCAAAACCTCATACCTGTCTCTTATACACATCTGACGCTGCCGACGA
+
;@@DB?B;CFBB#2<:CB:FH<C@:<A?C::#1:86:BG9:8?8688?888EBF;783)=6-7=CC;ECD);?7;;>>AE;>(5;->AC@;B@;8?#####
@SRR1363257.77 GWZHISEQ01:153:C1W31ACXX:5:1101:5069:2307 length=101
ATAATAGGCAATCGCGTCGGAACAGTTACCGGCCAAAGAGAGGCAGGGACTTAACGGCATGATGGTGACCTCAGTTAAGAGAAGCCTGTCTCTTATACACA


## Data analysis <a class="anchor" id="second-bullet"></a>

About quality scores in Illumina sequencing http://drive5.com/usearch/manual/quality_score.html

Initial number of lines:
- 1823504 amp_res_2.fastq
- 1823504 amp_res_1.fastq

4 lines for each reads -> 455876 reads

In [15]:
!wc -l data/amp_res_2.fastq
!wc -l data/amp_res_1.fastq

 1823504 data/amp_res_2.fastq
 1823504 data/amp_res_1.fastq


In [16]:
!grep 'length' data/amp_res_2.fastq | wc -l 

  455876


Installation fastqc. It makes an analysis of Illumina sequencing and it quality. About analysis results https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/

In [23]:
!brew install fastqc

For help with functions 

In [17]:
!fastqc -h

In [79]:
!mkdir fastqc_result

In [18]:
!fastqc -o fastqc_result /Users/a_nugmanova/study/IB/practic/Project1/data/amp_res_1.fastq /Users/a_nugmanova/study/IB/practic/Project1/data/amp_res_2.fastq 


Unzip and check the result. There must be several files in each directory.

In [20]:
!unzip fastqc_result/amp_res_1_fastqc.zip -d fastqc_result
!ls fastqc_result/amp_res_1_fastqc
!unzip fastqc_result/amp_res_2_fastqc.zip -d fastqc_result
!ls fastqc_result/amp_res_2_fastqc

In [24]:
!open fastqc_result/amp_res_1_fastqc.html

In [22]:
!open fastqc_result/amp_res_2_fastqc.html

## Filtering reads <a class="anchor" id="third-bullet"></a> 

With Trimmomatic we can cut read based on it quality. 

In [25]:
!brew install Trimmomatic

In [27]:
!ls data

In [28]:
!mkdir trimmomatic_result

Filtering parameters:
- ILLUMINACLIP: Cut adapter and other illumina-specific sequences from the read.
- SLIDINGWINDOW: Perform a sliding window trimming, cutting once the average quality within the window falls below a threshold.
- LEADING: Cut bases off the start of a read, if below a threshold quality
- TRAILING: Cut bases off the end of a read, if below a threshold quality
- CROP: Cut the read to a specified length
- HEADCROP: Cut the specified number of bases from the start of the read
- MINLEN: Drop the read if it is below a specified length
- TOPHRED33: Convert quality scores to Phred-33
- TOPHRED64: Convert quality scores to Phred-64

Full description http://www.usadellab.org/cms/?page=trimmomatic

Our parameters:
- Cut bases off the start of a read if quality below 20
- Cut bases off the end of a read if quality below 20
- Trim reads using a sliding window approach, with window size 10 and average quality within the window 20.
- Drop the read if it is below length 20.

In [29]:
!java -jar trimmomatic-0.39.jar PE data/amp_res_1.fastq data/amp_res_2.fastq trimmomatic_result/amp_res_1P.fq trimmomatic_result/amp_res_1U.fq  trimmomatic_result/amp_res_2P.fq trimmomatic_result/amp_res_2U.fq SLIDINGWINDOW:10:20 LEADING:20 TRAILING:20 MINLEN:20
                                    

amp_res_1P.fq contain forward paired reads  

amp_res_2P.fq contain forward unpaired reads  

amp_res_1U.fq contain reverse paired reads  

amp_res_2U.fq contain reverse unpaired reads  

In [30]:
!wc -l trimmomatic_result/amp_res_1P.fq
!wc -l trimmomatic_result/amp_res_2P.fq

 1782756 trimmomatic_result/amp_res_1P.fq
 1782756 trimmomatic_result/amp_res_2P.fq


In [31]:
!wc -l trimmomatic_result/amp_res_1U.fq
!wc -l trimmomatic_result/amp_res_2U.fq

   39032 trimmomatic_result/amp_res_1U.fq
    1136 trimmomatic_result/amp_res_2U.fq


After filtering we can check the quality again.

In [32]:
!fastqc -o fastqc_result /Users/a_nugmanova/study/IB/practic/Project1/trimmomatic_result/amp_res_1P.fq /Users/a_nugmanova/study/IB/practic/Project1/trimmomatic_result/amp_res_2P.fq


In [33]:
!unzip fastqc_result/amp_res_1P_fastqc.zip -d fastqc_result
!ls fastqc_result/amp_res_1P_fastqc
!unzip fastqc_result/amp_res_2P_fastqc.zip -d fastqc_result
!ls fastqc_result/amp_res_2P_fastqc

In [34]:
!open fastqc_result/amp_res_1P_fastqc.html

## Aligment  <a class="anchor" id="fourth-bullet"></a> 

BWA MEM algorithm for aligment. It needs index the reference sequence and strain sequence with forward and reverse. Returns sam file with aligment.

In [35]:
!brew install bwa
!bwa

In [36]:
!bwa index data/GCF_000005845.2_ASM584v2_genomic.fna 

In [37]:
!mkdir alignment

Aligment of source reads 

In [38]:
# !bwa mem data/GCF_000005845.2_ASM584v2_genomic.fna data/amp_res_1.fastq data/amp_res_2.fastq > alignment/alignment.sam


Aligment of filtered reads 

In [39]:
!bwa mem data/GCF_000005845.2_ASM584v2_genomic.fna trimmomatic_result/amp_res_1P.fq trimmomatic_result/amp_res_2P.fq > alignment/alignment.sam


About sam format https://samtools.github.io/hts-specs/SAMv1.pdf

In [51]:
!head -5 alignment/alignment.sam

@SQ	SN:NC_000913.3	LN:4641652
@PG	ID:bwa	PN:bwa	VN:0.7.17-r1188	CL:bwa mem data/GCF_000005845.2_ASM584v2_genomic.fna trimmomatic_result/amp_res_1P.fq trimmomatic_result/amp_res_2P.fq
SRR1363257.37	99	NC_000913.3	232649	60	94M	=	232767	190	GGTTGCAGATTCGCAGTGTCGCTGTTCCAGCGCATCACATCTTTGATGTTCACGCCGTGGCGTTTAGCAATGCTTGAAAGCGAATCGCCTTTGC	@?:=:;DBFADH;CAECEE@@E:FFHGAE4?C?DE<BFGEC>?>FHE4BFFIIFHIBABEECA83;>>@>@CCCDC9@@CC08<@?@BB@9:CC	NM:i:0	MD:Z:94	MC:Z:72M	AS:i:94	XS:i:0
SRR1363257.37	147	NC_000913.3	232767	60	72M	=	232649	-190	TGTTGTTTGCCAGTCGCTGTGCGCTACTGCCTGCACCAATCGTCAAACTTTGGCCTGGCTTCAGCTTAGATC	99A?A?5;..'8DHHE9AA8F<<8D?F;@>D@??:B:9@EGHHFHBA;HGFA;A+;;9@DFBHFD:BDB???	NM:i:0	MD:Z:72	MC:Z:94M	AS:i:72	XS:i:0
SRR1363257.46	99	NC_000913.3	1197462	60	41M	=	1197462	68	GTATGAGGTTTTGCTGCATTCTCTGNGCGAATATTAACTCC	@@<?=D?D==?<AFGDF+AIHEACH#22<:?E8??:9??GG	NM:i:1	MD:Z:25A15	MC:Z:28S68M	AS:i:39	XS:i:0


## Sort and index BAM file <a class="anchor" id="fifth-bullet"></a> 

BAM file is just compressed SAM file

In [40]:
!brew install samtools

In [41]:
!samtools view -S -b alignment/alignment.sam > alignment/alignment.bam

In [42]:
!samtools flagstat alignment/alignment.bam

891635 + 0 in total (QC-passed reads + QC-failed reads)
891378 + 0 primary
0 + 0 secondary
257 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
890569 + 0 mapped (99.88% : N/A)
890312 + 0 primary mapped (99.88% : N/A)
891378 + 0 paired in sequencing
445689 + 0 read1
445689 + 0 read2
887530 + 0 properly paired (99.57% : N/A)
889384 + 0 with itself and mate mapped
928 + 0 singletons (0.10% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [43]:
!samtools sort alignment/alignment.bam -o alignment/alignment_sorted.bam

In [44]:
!samtools index alignment/alignment_sorted.bam

## Variant finding <a class="anchor" id="sixth-bullet"></a> 

Mpileup goes through each position in reference sequence and compares it with reads.

In [45]:
!samtools mpileup -f data/GCF_000005845.2_ASM584v2_genomic.fna alignment/alignment_sorted.bam > my.mpileup

In [55]:
!head -5 my.mpileup

NC_000913.3	1	A	5	^].^].^],^],^],	>;EEB
NC_000913.3	2	G	7	...,,,,	E1=JII0
NC_000913.3	3	C	7	...,,,,	H9=IGH8
NC_000913.3	4	T	7	...,,,,	E;CGCD9
NC_000913.3	5	T	7	...,,,,	C<EIJJ:


VarScan select variants from mpileup file 

Source jar file: https://github.com/dkoboldt/varscan

In [46]:
!java -jar VarScan.v2.4.0.jar mpileup2snp -h

In [47]:
!java -jar varscan/VarScan.v2.4.0.jar mpileup2snp my.mpileup --min-var-freq 0.2 --variants --output-vcf 1 > VarScan_results.vcf


Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.2
Min avg qual:	15
P-value thresh:	0.01
Reading input from my.mpileup
4641476 bases in pileup file
9 variant positions (6 SNP, 3 indel)
0 were failed by the strand-filter
6 variant positions reported (6 SNP, 0 indel)


In [48]:
!grep -v "^#" VarScan_results.vcf | cut -f 1 | uniq

NC_000913.3


In [49]:
# !mkdir -p data/k12

## Variants annotation  <a class="anchor" id="seventh-bullet"></a> 

snpEff annotate all variants and return the information about changes, gene names etc.

Source jar file: http://pcingola.github.io/SnpEff/

In [60]:
!cp data/GCF_000005845.2_ASM584v2_genomic.gbff snpEff/data/ecoli_K12/genes.gbk
!ls snpEff

We need to build our own database of ecoli. 

Important moments:
- data/ecoli_K12/genes.gbk should be in directory with snpEff.jar
- near the snpEff.jar should be snpEff.config which include only(!) one line:
```ecoli_K12.genome : ecoli_K12```

In [56]:
!java -jar snpEff/snpEff.jar build -genbank -v ecoli_K12

In [61]:
!java -jar snpEff/snpEff.jar ann ecoli_K12 VarScan_results.vcf > VarScan_results_annotated.vcf

In [72]:
# !head -20 VarScan_results_annotated.vcf
!sed -n 25,30p VarScan_results_annotated.vcf

##SnpEffCmd="SnpEff  ecoli_K12 VarScan_results.vcf "
##INFO=<ID=ANN,Number=.,Type=String,Description="Functional annotations: 'Allele | Annotation | Annotation_Impact | Gene_Name | Gene_ID | Feature_Type | Feature_ID | Transcript_BioType | Rank | HGVS.c | HGVS.p | cDNA.pos / cDNA.length | CDS.pos / CDS.length | AA.pos / AA.length | Distance | ERRORS / WARNINGS / INFO' ">
##INFO=<ID=LOF,Number=.,Type=String,Description="Predicted loss of function effects for this variant. Format: 'Gene_Name | Gene_ID | Number_of_transcripts_in_gene | Percent_of_transcripts_affected'">
##INFO=<ID=NMD,Number=.,Type=String,Description="Predicted nonsense mediated decay effects for this variant. Format: 'Gene_Name | Gene_ID | Number_of_transcripts_in_gene | Percent_of_transcripts_affected'">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	Sample1
NC_000913.3	93043	.	C	G	.	PASS	ADP=17;WT=0;HET=0;HOM=1;NC=0;ANN=G|missense_variant|MODERATE|ftsI|b0084|transcript|b0084|protein_coding|1/1|c.1631C>G|p.Ala544Gly|1631